In [13]:
from numpy import *
import pandas as pd
import matplotlib.pyplot as plt
import scipy as sp
import time
#magnetization, here index is tempareture
Mdata = pd.read_csv('C:/Users/mikku/OneDrive/デスクトップ/研究大学院/解析/BrilluinFitting/data/M_Temperature.dat',sep='\t',names=['Mr','MFe'],skiprows=1)
PPdata = pd.read_csv('C:/Users/mikku/OneDrive/デスクトップ/研究大学院/解析/BrilluinFitting/data/pp_experiment',sep='\t',names=['LF_PP','HF_PP'],skiprows=1)
#GHz→Hzに変換
PPdata = PPdata * 10**9
PPdata

,LF_PP,HF_PP
6,1.795390e+10,4.536620e+11
20,1.204530e+10,4.442250e+11
30,1.125620e+10,4.357660e+11
35,1.057280e+10,4.168650e+11
40,9.204510e+09,3.924560e+11
43,9.538100e+09,3.721310e+11
45,1.197270e+10,3.587920e+11
48,8.772620e+09,3.354910e+11
50,8.432510e+09,3.206720e+11
53,8.940410e+09,2.968050e+11


In [14]:
def omega_Kriche(T,K,kai):
    H     = 2000               #external field
    gamma = 1.76 * 10**7       #gyromagnetic ratio
    m     = abs(Mdata.loc[T,'Mr']) - abs(Mdata.loc[T,'MFe'])
    M     = abs(Mdata.loc[T,'Mr']) + abs(Mdata.loc[T,'MFe'])
    #kai   = M**2/(lamb*abs(Mdata.loc[T,'Mr'])*abs(Mdata.loc[T,'MFe']))        #suspectibility    3.7 * 10**(-4)
    #print(kai)
    omega_0  = gamma * sqrt(2*K/kai)
    omega_KK = gamma * abs(m) / kai
    omega_H  = gamma * H
    Omega_3  = omega_H - omega_KK/2
    Omega_4  = sqrt(omega_KK**2 - 2*omega_0**2)/2
    return sqrt(Omega_3**2 + Omega_4**2 + sqrt(omega_0**4 + 16*Omega_3**2*Omega_4**2)/2) ,sqrt(Omega_3**2 + Omega_4**2 - sqrt(omega_0**4 + 16*Omega_3**2*Omega_4**2)/2)

In [15]:
#calculate residual
T_PP = PPdata.index.values
K_lim = linspace(5000,12000,50)
lamb_lim =linspace(1e-5,1e-1,100000)
res_sum = inf
res_list = []
count = 0
start_time = time.time()
K_second,lamb_second = 0,0
K_best,lamb_best = 0,0
temp = 0
for K in K_lim:
    for lamb in lamb_lim:
        if count%10000 == 0:
            print(f"time: {time.time()-start_time},count: {count},res_sum: {res_sum}")
            print(f"temp:{temp} K: {K},lamb: {lamb}")
        temp = 0
        count += 1
        for t in T_PP:
            HF,LF = omega_Kriche(t,K,lamb)
            temp = temp + (HF - PPdata.loc[t,'HF_PP'])**2 # + (LF - PPdata.loc[t,'LF_PP'])**2
        res_list.append(temp)
        if temp < res_sum:
            res_sum = temp
            K_third,lamb_third = K_second,lamb_second
            K_second,lamb_second = K_best,lamb_best
            K_best,lamb_best = K,lamb

time: 0.0,count: 0,res_sum: inf
temp:0 K: 5000.0,lamb: 1e-05
time: 13.087478637695312,count: 10000,res_sum: 1.621888832018096e+23
temp:9.084195037036667e+23 K: 5000.0,lamb: 0.010009099990999911


C:\Users\mikku\AppData\Local\Temp\ipykernel_24652\631338914.py:13: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(Omega_3**2 + Omega_4**2 + sqrt(omega_0**4 + 16*Omega_3**2*Omega_4**2)/2) ,sqrt(Omega_3**2 + Omega_4**2 - sqrt(omega_0**4 + 16*Omega_3**2*Omega_4**2)/2)


time: 26.109689474105835,count: 20000,res_sum: 1.621888832018096e+23
temp:1.7167601744935858e+24 K: 5000.0,lamb: 0.020008199981999823
time: 39.09027695655823,count: 30000,res_sum: 1.621888832018096e+23
temp:1.9408543110472665e+24 K: 5000.0,lamb: 0.03000729997299973
time: 52.432122468948364,count: 40000,res_sum: 1.621888832018096e+23
temp:2.0056937726155732e+24 K: 5000.0,lamb: 0.04000639996399965
time: 66.13177394866943,count: 50000,res_sum: 1.621888832018096e+23
temp:2.0270113019160723e+24 K: 5000.0,lamb: 0.05000549995499956
time: 80.50193119049072,count: 60000,res_sum: 1.621888832018096e+23
temp:2.0283005845158541e+24 K: 5000.0,lamb: 0.060004599945999465


C:\Users\mikku\AppData\Local\Temp\ipykernel_24652\631338914.py:12: RuntimeWarning: invalid value encountered in sqrt
  Omega_4  = sqrt(omega_KK**2 - 2*omega_0**2)/2


time: 95.06488418579102,count: 70000,res_sum: 1.621888832018096e+23
temp:nan K: 5000.0,lamb: 0.07000369993699937
time: 108.45504093170166,count: 80000,res_sum: 1.621888832018096e+23
temp:nan K: 5000.0,lamb: 0.08000279992799929
time: 121.85171866416931,count: 90000,res_sum: 1.621888832018096e+23
temp:nan K: 5000.0,lamb: 0.09000189991899919
time: 135.46188163757324,count: 100000,res_sum: 1.621888832018096e+23
temp:nan K: 5142.857142857143,lamb: 1e-05
time: 149.0576536655426,count: 110000,res_sum: 1.621888832018096e+23
temp:9.087117429408114e+23 K: 5142.857142857143,lamb: 0.010009099990999911
time: 162.77048301696777,count: 120000,res_sum: 1.621888832018096e+23
temp:1.7171440557856973e+24 K: 5142.857142857143,lamb: 0.020008199981999823
time: 176.18284249305725,count: 130000,res_sum: 1.621888832018096e+23
temp:1.9412999945988902e+24 K: 5142.857142857143,lamb: 0.03000729997299973
time: 189.21978616714478,count: 140000,res_sum: 1.621888832018096e+23
temp:2.006191401261941e+24 K: 5142.8571428

In [ ]:

print(res_sum,K_best,lamb_best)
print(min(res_list))

2.0301329323616855e+24 5e-05 0.1
2.0301329323616855e+24
